# Creating a new method in Agent

This notebook will take you through all the steps required to create an LLM prompting method from scratch, so that you can do the same for your custom methods. We will look at creating everything needed for the direct method. This method is already a part of the framework but we will recreate it from scratch so that you can follow the same steps for your own methods.

Note: Please make sure you have first installed all dependencies of Agent, following the installation guide!

## Step 1: Create prompt templates

This is similar to the first step in creating a new task, since most tasks require their own definition of prompting methods to fit the environment. in this exmaple we create the direct method for the GSM8k task.

Under `../src/agent/prompts/templates/gsm8k/` we will define the template .jinja files for any messages sent to the LLM according to our new method. 

All methods must end with an external action which is taken in the task environment. As such an `external_action.jinja` file must be defined in the appropriate task folder (or the `../src/agent/prompts/templates/default/external_action.jinja` can be extended). 

In fact, for the direct prompting method, onlt the external action template is necessary:

In [ ]:
external_action = """
[[ SYSTEM ]]
You are an expert mathematician. You are provided with mathematical questions that you have to answer.
When asked for an answer, your response should use the following format:
Answer: <answer>

[[ USER ]]
Question: {{memory.retrieve(memory.mem_keys.OBSERVATION})}}

Now please answer the question.
Answer in the format
Answer: <answer>
"""

with open("../src/agent/prompts/templates/gsm8k/external_action.jinja", "w") as file:
    file.write(external_action)

Here we defined all the template content needed for the direct prompting emthod from scratch, but actually you can often use content from the `../src/agent/prompts/templates/default/` templates folder, and even task templates such as `../src/agent/prompts/templates/gsm8k/system_prompt.jinja`, which are not method-specific. Many methods will reuse a lot of the same prompt content, and as such we enable users to simply define the new elements that change, as seen in the "Create a new task" guide. Also note that while the direct prompting method only requires an external action template for prompting the LLM, methods which prompt the LLM multiple times will require at least one template per different call.

## Step 2: Create a method configuration

Create a new configuration file under `../configs/method/` which should define the method's Flow. See the documentation for more details on Flows and intrinsic functions. Here we use already existing flows and intrinsic functions to define the direct prompting method, composed of a single `SequentialFlow` with a single `Act` command:

In [ ]:
direct = """
# @package _global_
agent:
  pre_action_flow:
    _target_: agent.commands.SequentialFlow
    sequence:
      - _target_: agent.commands.Act
"""

with open("../configs/method/direct_example.jinja", "w") as file:
    file.write(direct)

## Step 3: Running the method

Assuming you are using an existing LLM configuration, say OpenChat-3.5, the method can be run for GSM8k as follows. The method argument must be the name of the config file defined in [Step 2](#step-2-create-a-method-configuration).

In [ ]:
! python ../src/agent/start.py task=gsm8k method=direct llm@agent.llm=hf/openchat_3.5 max_episodes=3

## Clean up files

Here we clean up the files created by this notebook to avoid cluttering the framework.

In [ ]:
import os

try:
    os.remove("../src/agent/prompts/templates/gsm8k/external_action.jinja")
    os.remove("../configs/method/direct_example.jinja")
except OSError as e:
    print(e)